In [22]:
import numpy as np
import cv2

In [23]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png').astype(np.float32) 
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [28]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    # ------------TODO --------------
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    h, w = depth.shape
    u = np.arange(w)
    v = np.arange(h)
    u, v = np.meshgrid(u, v)
    
    Z = depth
    X = (u - cx) * Z / fx
    Y = (v - cy) * Z / fy
    
    idx = np.where(seg == 255)
    pc = np.array([X[idx].flatten(), Y[idx].flatten(), Z[idx].flatten()]).T
    
    return pc

partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

In [34]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

# -----------TODO---------------
# implement one way chamfer distance
# -----------TODO---------------
one_way_CD = 0
partial_pc_sampled = partial_pc_sampled[:, np.newaxis]
full_pc_sampled = full_pc_sampled[np.newaxis]
dis = np.linalg.norm(partial_pc_sampled - full_pc_sampled, axis=-1)
one_way_CD = np.array([np.mean(np.min(dis, axis=1), axis=0)])
print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', one_way_CD)

one way chamfer distance:  [0.0099271]
